## Prompt Lab Challenge Exercises Notebook

프롬프트 랩 챌린지 연습 노트북에 오신 걸 환영합니다! 부트캠프 시리즈의 두 번째 실습입니다. 실습 1의 연습을 모두 완료했다면, 여기서의 대부분의 연습이 수월할 거에요.

이 노트북은 연습은 프롬프트가 잘 작동했을 때 어떤 결과가 나와야 하는지 표시된 템플릿이에요.

먼저 시작하기 전에, 이전 실습에서 말한대로 필요한 라이브러리가 설치된 Python 환경과 다음 내용이 담긴 .env 파일이 필요해요.

+ IBM Cloud API 키  
+ IBM Cloud 지역 URL (예: https://us-south.ml.cloud.ibm.com)  
+ WatsonX 프로젝트에 연결된 프로젝트 ID (WML Python SDK에서 필요)  

이 연습을 완료하는 데는 약 30~45분이 소요될 것으로 예상돼요. 편안하게 진행해 보세요!  

행운을 빕니다. 답을 비교해보는 걸 잊지 마세요!







In [1]:
# 한국어 요약 , in Korean:, han-gul: 등이있다.
import os
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

In [2]:
#config Watsonx.ai environment
load_dotenv()
api_key = os.getenv("API_KEY", None)
ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None)
project_id = os.getenv("PROJECT_ID", None)
if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }


In [3]:
def send_to_watsonxai(prompts,
                    model_name="meta-llama/llama-2-70b-chat",
                    decoding_method="greedy",
                    max_new_tokens=100,
                    min_new_tokens=30,
                    temperature=1.0,
                    repetition_penalty=1.0,
                    stop_sequence=['\n\n']
                    ):
    '''
   helper function for sending prompts and params to Watsonx.ai
    
    Args:  
        prompts:list list of text prompts
        decoding:str Watsonx.ai parameter "sample" or "greedy"
        max_new_tok:int Watsonx.ai parameter for max new tokens/response returned
        temperature:float Watsonx.ai parameter for temperature (range 0>2)
        repetition_penalty:float Watsonx.ai parameter for repetition penalty (range 1.0 to 2.0)

    Returns: None
        prints response
    '''

    assert not any(map(lambda prompt: len(prompt) < 1, prompts)), "make sure none of the prompts in the inputs prompts are empty"

    # Instantiate parameters for text generation
    model_params = {
        GenParams.DECODING_METHOD: decoding_method,
        GenParams.MIN_NEW_TOKENS: min_new_tokens,
        GenParams.MAX_NEW_TOKENS: max_new_tokens,
        GenParams.RANDOM_SEED: 42,
        GenParams.TEMPERATURE: temperature,
        GenParams.REPETITION_PENALTY: repetition_penalty,
    }
    
    # Instantiate a model proxy object to send your requests
    model = Model(
        model_id=model_name,
        params=model_params,
        credentials=creds,
        project_id=project_id)


    for prompt in prompts:
        print(model.generate_text(prompt))


In [4]:
FLAN_T5_XXL = 'google/flan-t5-xxl'
FLAN_UL2 = 'google/flan-ul2'
GPT_NEOX = 'eleutherai/gpt-neox-20b'
GRANITE_13B_CHAT = 'ibm/granite-13b-chat-v1'
GRANITE_13B_INSTRUCT = 'ibm/granite-13b-instruct-v1'
LLAMA_2_70B_CHAT = 'meta-llama/llama-2-70b-chat'
MPT_7B_INSTRUCT2 = 'ibm/mpt-7b-instruct2'
MT0_XXL = 'bigscience/mt0-xxl'
STARCODER = 'bigcode/starcoder'

#### Q1) 리뷰의 감정을 반환하기 위한 프롬프트를 작성하세요.
Target sentiment = positive

In [5]:
# Product Review for Questions  1 - 5
review = """침실에 좋은 램프가 필요해서 이 제품을 선택했는데, 추가 저장 공간이 있고 가격도 너무 싸요. 
빨리 받았습니다. 우리 램프의 스트링이 운송 중에 끊겼는데, 회사에서 기꺼이 새로운 것을 보내주었어요. 
또한 몇 일 안에 도착했습니다. 조립하기 쉬웠어요. 
부품이 빠져 있어 고객 지원팀에 연락했고, 빠르게 보내주었습니다! 
Lumina는 고객과 제품에 관심을 가지고 있는 회사로 보입니다!!"""


In [6]:
#Q1 Code - enter prompt and parameters in this cell
prompt = f"""
{review}

위의 문장의 감정을 positive 또는 negative로  분류해줘. 
"""
#영어를 사용하지말고 한국어를 사용하세요 #Complete your prompt here 

response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_2_70B_CHAT, decoding_method="greedy", max_new_tokens=9,
                              min_new_tokens=1, temperature=0, repetition_penalty=1.0)



답: Positive


#### Q2) 리뷰어가 표현한 감정을 추출하고, 답변을 쉼표로 구분된 목록으로 반환하세요.
대상 감정 = 만족, 제품 좋음, 회사좋음 , 배송좋음

In [7]:

prompt = f''' 
"""{review}"""

삼중따옴표로 구성된 위의 문장의 감정을 만족, 불만족, 분노, 슬픔, 행복, 실망, 제품 좋음, 회사좋음, 배송좋음 중에서 추출해줘.
'''
#영어를 사용하지말고 한국어를 사용하세요 #Complete your prompt here 

response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_2_70B_CHAT,  decoding_method="greedy", max_new_tokens=100,
                              min_new_tokens=1, temperature=0, repetition_penalty=1.0)


답: 만족, 제품 좋음, 회사좋음, 배송좋음


#### Q3) 리뷰어가 분노를 표현하고 있는가요? "예" 또는 "아니오"로 답하세요. – 분노를 포함한 여러 예제를 사용하여 양쪽 경우에 모두 작동하는지 확인하세요.
Target answer = 아니요

In [8]:
prompt = f"""
'''{review}'''

위의 문장은 분노를 표현하고 있는가요? '예' 또는 '아니요'로 알려줘.
"""
response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_2_70B_CHAT,  decoding_method="greedy", max_new_tokens=10,
                              min_new_tokens=1, temperature=0, repetition_penalty=1.0)


답: 아니요.


#### Q4) 구매한 상품과 회사 이름을 추출하고 JSON 형식으로 반환하세요.
대상 답변 = product[침실 램프], name[Lumina]

In [9]:
prompt = f"""
'''{review}'''

위 단락을 기반으로 리뷰된 제품의 이름과 해당 제품을 판매하는 회사의 이름을 찾으세요.
찾은 이름을 사용하여 JSON을 작성하세요.

"""
response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_2_70B_CHAT,  decoding_method="greedy", max_new_tokens=33,
                              min_new_tokens=1, temperature=0, repetition_penalty=1.0)

{
"name": "Lumina",
"product": "침실 램프"
}




#### Q5) 여러가지 프롬프트로 결합하고 JSON으로 반환하세요
대상 답변 = positive[true], Anger[false], Brand[Lumina]








In [10]:
prompt = f"""
'''{review}'''

위 리뷰를 기반으로 다음 질문에 대한 답을 찾으세요.

리뷰에 긍정적인 감정이 있는지 여부를 찾으세요.
리뷰 작성자가 분노를 표현했는지 여부를 찾으세요.
리뷰된 제품을 판매하는 회사의 이름을 찾으세요.
이에 대한 답으로 JSON을 작성하세요.
"""
response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_2_70B_CHAT,  decoding_method="greedy", max_new_tokens=45,
                              min_new_tokens=1, temperature=0, repetition_penalty=1.0)


{
"positive_sentiment": true,
"angry": false,
"company_name": "Lumina"
}


In [11]:
review = "딸의 생일을 위해 이 판다 인형을 샀습니다. \
딸이 그것을 정말 좋아하고 어디든 가져다 놉니다. \
부드럽고 귀여워서 얼굴도 친근한 느낌이에요. \
그러나 내가 지불한 가격에 비해 조금 작은 편이에요. \
동일한 가격으로 더 큰 옵션이 있을거 같아요. \
예상보다 하루 일찍 도착해서 딸에게 선물하기 전에 제가 직접 놀 수 있었어요."


#### Q6) 다음 제품 리뷰를 요약하세요.
예시 요약 = 
* 딸의 생일 선물로 판다 인형을 샀는데, 딸이 정말 좋아하고 가져다 놉니다.
* 인형이 부드럽고 귀여워서 얼굴도 친근한 느낌이에요.
* 하지만 내가 지불한 가격에 비해 조금 작은 편이에요.

In [12]:
prompt = f"""
'''{review}'''

위의 리뷰를 기반으로 한국어 요약을 작성해주세요.
최대 3문장으로 작성해주세요.
"""

response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_2_70B_CHAT, decoding_method="greedy", max_new_tokens=200,
                              min_new_tokens=30, repetition_penalty=1.0)


* 딸의 생일 선물로 판다 인형을 샀는데, 딸이 정말 좋아하고 가져다 놉니다.
* 인형이 부드럽고 귀여워서 얼굴도 친근한 느낌이에요.
* 하지만 내가 지불한 가격에 비해 조금 작은 편이에요.


#### Q7) 동일한 제품 리뷰를 배송 부서의 관점에서 요약하라. 
예시 요약 = 
* 주문이 정시에 도착했습니다.
* 수령한 패키지가 손상되지 않았습니다.
* 예상보다 하루 일찍 도착해서 딸에게 선물하기 전에 제가 직접 놀 수 있었어요.




In [13]:
prompt = f"""
'''{review}'''

위의 리뷰를 기반으로 배송과 관련된 요약을 '한국어'로 작성해 보세요.
주문이 정시에 도착했는지의 여부, 수령한 패키지가 손상되었는지 여부 등이 있습니다.
최대 3문장으로 작성해주세요.
"""

response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_2_70B_CHAT, decoding_method="greedy", max_new_tokens=200,
                              min_new_tokens=30, repetition_penalty=1.0)


배송 요약:

* 주문이 정시에 도착하였습니다.
* 수령한 패키지가 손상되지 않았습니다.
* 예상보다 하루 일찍 도착해서 딸에게 선물하기 전에 제가 직접 놀 수 있었어요.


#### Q8) 개인 정보 식별 제거. 
다음 이메일이 주어지면 개인 식별 정보 (예: 이름, 이메일 등)를 제거하기 위한 프롬프트를 작성하십시오.







In [14]:
email="""
안녕하세요 존,

저는 최근에 새 차를 구입한 것을 알게 되어서 당신에게 편지를 씁니다. 
저는 지역 딜러쉽 (Cheap Dealz)에서 영업 사원으로 일하고 있으며 새 차에 대한 훌륭한 거래가 있다는 것을 알려드리고 싶었습니다. 
관심이 있으시면 알려주시기 바랍니다.

감사합니다,

전화: 410-805-2345
이메일: jimmysmith@cheapdealz.com"
"""

In [15]:
prompt = f"""
email: ```{email}```

위의 이메일을 기반으로 다음과 같이 사람 이름, 주소, 전화번호, 휴대폰 번호, 이메일, 신용카드 번호, 의료 정보 등 민감한 정보를 삭제하여 이메일을 다시 만들어주세요.

새 이메일:
"""
# response = send_to_watsonxai(prompts=[prompt])
# Remember to try changing the model and/or using non-default parameters to achieve your goal when prompting isn't enough
response = send_to_watsonxai(prompts=[prompt], model_name=LLAMA_2_70B_CHAT, decoding_method="greedy", max_new_tokens=235,
                              min_new_tokens=1, repetition_penalty=1.0, temperature=0)


안녕하세요,

저는 최근에 새 차를 구입한 것을 알게 되어서 당신에게 편지를 씁니다. 
저는 지역 딜러쉽 (Cheap Dealz)에서 영업 사원으로 일하고 있으며 새 차에 대한 훌륭한 거래가 있다는 것을 알려드리고 싶었습니다. 
관심이 있으시면 알려주시기 바랍니다.

감사합니다
